In [1]:
!nvidia-smi

Tue Jan 24 22:31:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.1 MB/s eta 0:00:00


In [5]:
import pandas as pd

import tensorflow as tf

import keras

import torch

from transformers import BertTokenizer, BertModel, BertForMaskedLM, pipeline

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sb

In [ ]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [ ]:
#files = [file,email,http,logon,device]

In [ ]:
#for x in files:
#    print(x.shape)
file.shape

(445581, 6)

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
named_params = list(model.named_parameters())

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
tokenizer.encode_plus('Ken is the best!')

{'input_ids': [101, 6358, 2003, 1996, 2190, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
response = model(torch.tensor(tokenizer.encode('Ken is the best')).unsqueeze(0))

In [ ]:
response

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1575,  0.1448, -0.0683,  ..., -0.2043,  0.2934,  0.2390],
         [ 0.0336, -0.8281, -0.0863,  ..., -0.4555,  0.7815,  0.0969],
         [-0.3097, -0.4105,  0.0555,  ...,  0.0817,  0.9685,  0.6817],
         [-0.6090, -0.9053, -0.0094,  ..., -0.2928,  1.2827, -0.1907],
         [-0.8572, -0.4660, -0.3183,  ...,  0.8195,  0.9794, -0.5100],
         [ 0.8410,  0.0964, -0.0524,  ...,  0.0601, -0.5761, -0.3895]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8167, -0.2347,  0.4112,  0.5842, -0.2368, -0.0547,  0.8562,  0.1426,
          0.2936, -0.9997,  0.1450,  0.0200,  0.9754, -0.2654,  0.9061, -0.5125,
         -0.0333, -0.4753,  0.2095, -0.6647,  0.5949,  0.9258,  0.6194,  0.2113,
          0.3049,  0.1423, -0.4651,  0.8916,  0.9368,  0.6442, -0.5724,  0.1174,
         -0.9778, -0.1278,  0.3968, -0.9710,  0.1264, -0.6902,  0.0774,  0.0675,
         -0.8322,  0.1926,  0.9963, -0.6426, -

In [ ]:
response.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [ ]:
cls_embedding = response.last_hidden_state[:, 0, :].unsqueeze(0)

In [ ]:
cls_embedding.shape

torch.Size([1, 1, 768])

In [ ]:
model.pooler(cls_embedding).shape

torch.Size([1, 768])

In [ ]:
(model.pooler(cls_embedding) == response.pooler_output).all()

tensor(True)

In [ ]:
'sinan' in tokenizer.vocab

False

In [ ]:
i=0

In [ ]:
while i < 10:
    encoded_file_content = tokenizer.encode(file['content'][0])

In [ ]:
file.size

2673486

In [ ]:
encoded_file_content[7]

[101,
 1040,
 2692,
 1011,
 12935,
 1011,
 2340,
 1011,
 1041,
 2692,
 1011,
 17350,
 1011,
 29491,
 1011,
 20720,
 1011,
 1041,
 2487,
 2350,
 2150,
 2116,
 2230,
 4265,
 3225,
 15175,
 4789,
 28740,
 2015,
 8198,
 18778,
 21662,
 2717,
 2125,
 4810,
 2070,
 2021,
 2456,
 2633,
 2757,
 5883,
 2570,
 1039,
 4198,
 5797,
 2952,
 9037,
 5550,
 16232,
 2207,
 14712,
 2207,
 2043,
 4653,
 2853,
 11397,
 4078,
 8586,
 9250,
 8494,
 3149,
 2332,
 2264,
 6368,
 2590,
 2612,
 2024,
 4746,
 2372,
 2789,
 6926,
 102]

In [ ]:
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
example = "Ken is the best guy!"

In [ ]:
tokenizer.encode(example, return_tensors='pt')

tensor([[ 101, 6358, 2003, 1996, 2190, 3124,  999,  102]])

In [ ]:
model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
         [-0.0098, -0.0581, -0.0190,  ..., -0.0698, -0.0208, -0.0665],
         [-0.0360, -0.0246, -0.0257,  ...,  0.0034, -0.0018,  0.0269],
         ...,
         [-0.0141, -0.1070,  0.0017,  ..., -0.0354, -0.0362, -0.0096],
         [ 0.0298, -0.0373, -0.0356,  ...,  0.0161,  0.0192,  0.0173],
         [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model.embeddings.position_embeddings

Embedding(512, 768)

In [ ]:
model.embeddings.position_embeddings(torch.LongTensor(range(9)))

tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 6.4257e-03, -1.7664e-02, -2.2067e-02,  ...,  6.7531e-04,
          1.1108e-02,  3.7521e-03],
        [ 6.2613e-04, -1.6089e-02, -7.6365e-03,  ...,  5.3390e-03,
          1.5909e-02,  1.8119e-03],
        [ 2.0350e-03, -1.4569e-02, -1.4560e-02,  ...,  1.4465e-02,
          1.0401e-02,  3.7587e-03]], grad_fn=<EmbeddingBackward0>)

In [ ]:
torch.LongTensor([0]*6)

tensor([0, 0, 0, 0, 0, 0])

In [ ]:
model.embeddings.token_type_embeddings(torch.LongTensor([0]*6))

tensor([[ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model.embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
model.embeddings.LayerNorm(
    model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt')) +
    model.embeddings.position_embeddings(torch.LongTensor(range(8))) +
    model.embeddings.token_type_embeddings(torch.LongTensor([0]*8))
)

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
model.embeddings(tokenizer.encode(example, return_tensors='pt')).size()

torch.Size([1, 8, 768])

In [7]:
bert_lm = BertForMaskedLM.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
bert_lm

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [10]:
nlp = pipeline('fill-mask', model = 'bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
type(nlp.model)

transformers.models.bert.modeling_bert.BertForMaskedLM

In [11]:
nlp.tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [16]:
preds = nlp(f"If you dont {nlp.tokenizer.mask_token} at the sign, you will get a ticket.")

In [17]:
for p in preds:
  print(f"token: {p['token_str']}")

token: look
token: stop
token: glance
token: turn
token: stare
